In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras

/home/sfefilatyev/projects/tensorflow_exercises/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sfefilatyev/projects/tensorflow_exercises/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sfefilatyev/projects/tensorflow_exercises/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int1

In [2]:
housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [3]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=X_train.shape[1:]),
    keras.layers.Dense(1)
])
model.compile(loss="mean_squared_error", optimizer="sgd")
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))
mse_test = model.evaluate(X_test, y_test)
X_new = X_test[:3] # pretend these are new instances
y_pred = model.predict(X_new)
print(y_pred)

Train on 11610 samples, validate on 3870 samples
Epoch 1/20
11610/11610 [==============================] - 1s 99us/sample - loss: 0.8779 - val_loss: 0.6241
Epoch 2/20
11610/11610 [==============================] - 0s 26us/sample - loss: 0.5491 - val_loss: 0.5057
Epoch 3/20
11610/11610 [==============================] - 0s 26us/sample - loss: 0.6182 - val_loss: 0.5068
Epoch 4/20
11610/11610 [==============================] - 0s 26us/sample - loss: 0.4914 - val_loss: 0.4687
Epoch 5/20
11610/11610 [==============================] - 0s 26us/sample - loss: 0.4666 - val_loss: 0.4520
Epoch 6/20
11610/11610 [==============================] - 0s 26us/sample - loss: 0.4556 - val_loss: 0.4502
Epoch 7/20
11610/11610 [==============================] - 0s 25us/sample - loss: 0.4475 - val_loss: 0.4313
Epoch 8/20
11610/11610 [==============================] - 0s 25us/sample - loss: 0.4377 - val_loss: 0.4235
Epoch 9/20
11610/11610 [==============================] - 0s 25us/sample - loss: 0.4355 - val_l

In [5]:
# Wide and deep neural net.
input_ = keras.layers.Input(shape=X_train.shape[1:])
hidden1 = keras.layers.Dense(30, activation="relu")(input_)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.Concatenate()([input_, hidden2])
output = keras.layers.Dense(1)(concat)
model2 = keras.Model(inputs=[input_], outputs=[output])
model2.compile(loss="mean_squared_error", optimizer="Adam")
print(model2.summary())
history2 = model2.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))
mse_test2 = model2.evaluate(X_test, y_test)
X_new = X_test[:3] # pretend these are new instances
y_pred = model2.predict(X_new)
print(y_pred)

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 8)]          0                                            
__________________________________________________________________________________________________
dense_5 (Dense)                 (None, 30)           270         input_2[0][0]                    
__________________________________________________________________________________________________
dense_6 (Dense)                 (None, 30)           930         dense_5[0][0]                    
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 38)           0           input_2[0][0]                    
                                                                 dense_6[0][0]              

In [14]:
# Wide and deep network with auxilliary output
input_A = keras.layers.Input(shape=[5], name="wide_input")
input_B = keras.layers.Input(shape=[6], name="deep_input")
hidden1 = keras.layers.Dense(30, activation="relu")(input_B)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.concatenate([input_A, hidden2])
output = keras.layers.Dense(1, name="output")(concat)
model3 = keras.Model(inputs=[input_A, input_B], outputs=[output])
model3.compile(loss="mse", optimizer="Adam") # optimizer=keras.optimizers.SGD(lr=1e-3))

X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]
history = model3.fit((X_train_A, X_train_B), y_train, epochs=20,
                    validation_data=((X_valid_A, X_valid_B), y_valid))
mse_test = model3.evaluate((X_test_A, X_test_B), y_test)
y_pred = model3.predict((X_new_A, X_new_B))

Train on 11610 samples, validate on 3870 samples
Epoch 1/20
11610/11610 [==============================] - 0s 42us/sample - loss: 1.3032 - val_loss: 0.5476
Epoch 2/20
11610/11610 [==============================] - 0s 30us/sample - loss: 0.5012 - val_loss: 0.4541
Epoch 3/20
11610/11610 [==============================] - 0s 30us/sample - loss: 0.4433 - val_loss: 0.4154
Epoch 4/20
11610/11610 [==============================] - 0s 29us/sample - loss: 0.4198 - val_loss: 0.3935
Epoch 5/20
11610/11610 [==============================] - 0s 30us/sample - loss: 0.4034 - val_loss: 0.3880
Epoch 6/20
11610/11610 [==============================] - 0s 30us/sample - loss: 0.3952 - val_loss: 0.3952
Epoch 7/20
11610/11610 [==============================] - 0s 30us/sample - loss: 0.3831 - val_loss: 0.3731
Epoch 8/20
11610/11610 [==============================] - 0s 30us/sample - loss: 0.3782 - val_loss: 0.3640
Epoch 9/20
11610/11610 [==============================] - 0s 30us/sample - loss: 0.3763 - val_l

In [7]:
model3.layers

In [8]:
model3.layers[1]

In [9]:
hidden1 = model3.layers[1]

In [10]:
hidden1.name

'dense_8'

In [11]:
model3.get_layer('dense_8') is hidden1

True

In [12]:
# Saving and restoring model
import os
if os.path.exists("model4.hdf5"):
    model4 = keras.models.load_model("model4.hdf5")
else:
    main_output = keras.layers.Dense(1, name="main_output")(concat)
    aux_output = keras.layers.Dense(1, name="aux_output")(hidden2)
    model4 = keras.Model(inputs=[input_A, input_B], outputs=[main_output, aux_output])
    model4.compile(loss=["mse", "mse"], loss_weights=[0.9, 0.1], optimizer=keras.optimizers.SGD(lr=1e-3))

    history = model4.fit((X_train_A, X_train_B), [y_train, y_train], epochs=20,
                        validation_data=((X_valid_A, X_valid_B), [y_valid, y_valid]))
total_loss, main_loss, aux_loss = model4.evaluate([X_test_A, X_test_B], [y_test, y_test])
y_pred_main, y_pred_aux = model4.predict([X_new_A, X_new_B])
print(total_loss, main_loss, aux_loss)
print(y_pred_main, y_pred_aux)
if not os.path.exists("model4.hdf5"):
    model4.save("model4.hdf5")

5160/5160 [==============================] - 0s 28us/sample - loss: 0.5064 - main_output_loss: 0.4723 - aux_output_loss: 0.8139
0.5064139545425889 0.47233632 0.8139463
[[0.8056086]
 [1.8225387]
 [1.5147371]] [[1.2959085]
 [2.042921 ]
 [1.7641346]]


In [13]:
# Subclassing-API model
class WideAndDeepModel(keras.Model):
    def __init__(self, units=30, activation="relu", **kwargs):
        super().__init__(**kwargs) # handles standard args (e.g., name)
        self.hidden1 = keras.layers.Dense(units, activation=activation)
        self.hidden2 = keras.layers.Dense(units, activation=activation)
        self.main_output = keras.layers.Dense(1)
        self.aux_output = keras.layers.Dense(1)

    def call(self, inputs):
        input_A, input_B = inputs
        hidden1 = self.hidden1(input_B)
        hidden2 = self.hidden2(hidden1)
        concat = keras.layers.concatenate([input_A, hidden2])
        main_output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        return main_output, aux_output

model5 = WideAndDeepModel()
model5.compile(loss=["mse", "mse"], loss_weights=[0.9, 0.1], optimizer=keras.optimizers.SGD(lr=1e-3))
main_output, aux_output = model5(inputs=[input_A, input_B])
history = model5.fit((X_train_A, X_train_B), [y_train, y_train], epochs=20,
                     validation_data=((X_valid_A, X_valid_B), [y_valid, y_valid]))
total_loss, main_loss, aux_loss = model5.evaluate([X_test_A, X_test_B], [y_test, y_test])
y_pred_main, y_pred_aux = model5.predict([X_new_A, X_new_B])
print(total_loss, main_loss, aux_loss)
print(y_pred_main, y_pred_aux)

Train on 11610 samples, validate on 3870 samples
Epoch 1/20
11610/11610 [==============================] - 1s 50us/sample - loss: 3.2018 - output_1_loss: 2.9056 - output_2_loss: 5.8599 - val_loss: 1.7419 - val_output_1_loss: 1.3326 - val_output_2_loss: 5.4234
Epoch 2/20
11610/11610 [==============================] - 0s 32us/sample - loss: 1.4021 - output_1_loss: 1.0525 - output_2_loss: 4.5475 - val_loss: 1.1306 - val_output_1_loss: 0.8265 - val_output_2_loss: 3.8666
Epoch 3/20
11610/11610 [==============================] - 0s 33us/sample - loss: 0.9926 - output_1_loss: 0.7423 - output_2_loss: 3.2456 - val_loss: 0.9116 - val_output_1_loss: 0.7001 - val_output_2_loss: 2.8179
Epoch 4/20
11610/11610 [==============================] - 0s 32us/sample - loss: 0.8476 - output_1_loss: 0.6667 - output_2_loss: 2.4747 - val_loss: 0.8051 - val_output_1_loss: 0.6447 - val_output_2_loss: 2.2481
Epoch 5/20
11610/11610 [==============================] - 0s 31us/sample - loss: 0.7734 - output_1_loss: 0.

In [15]:
# Using callbacks
input_A = keras.layers.Input(shape=[5], name="wide_input")
input_B = keras.layers.Input(shape=[6], name="deep_input")
hidden1 = keras.layers.Dense(30, activation="relu")(input_B)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.concatenate([input_A, hidden2])
output = keras.layers.Dense(1, name="output")(concat)
model6 = keras.Model(inputs=[input_A, input_B], outputs=[output])
model6.compile(loss="mse", optimizer="Adam")

checkpoint_cb = keras.callbacks.ModelCheckpoint("model6.h5", save_best_only=True)

X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]
history = model6.fit((X_train_A, X_train_B), y_train, epochs=20,
                    validation_data=((X_valid_A, X_valid_B), y_valid), callbacks=[checkpoint_cb])
mse_test = model6.evaluate((X_test_A, X_test_B), y_test)
y_pred = model6.predict((X_new_A, X_new_B))

Train on 11610 samples, validate on 3870 samples
Epoch 1/20
11610/11610 [==============================] - 1s 43us/sample - loss: 1.7408 - val_loss: 0.7575
Epoch 2/20
11610/11610 [==============================] - 0s 31us/sample - loss: 0.7029 - val_loss: 0.5213
Epoch 3/20
11610/11610 [==============================] - 0s 36us/sample - loss: 0.5171 - val_loss: 0.4578
Epoch 4/20
11610/11610 [==============================] - 0s 31us/sample - loss: 0.4521 - val_loss: 0.4123
Epoch 5/20
11610/11610 [==============================] - 0s 34us/sample - loss: 0.4120 - val_loss: 0.3845
Epoch 6/20
11610/11610 [==============================] - 0s 33us/sample - loss: 0.3861 - val_loss: 0.3637
Epoch 7/20
11610/11610 [==============================] - 0s 31us/sample - loss: 0.3708 - val_loss: 0.3561
Epoch 8/20
11610/11610 [==============================] - 0s 30us/sample - loss: 0.3593 - val_loss: 0.3417
Epoch 9/20
11610/11610 [==============================] - 0s 30us/sample - loss: 0.3510 - val_l